In [1]:
using Revise
using Test

In [2]:
includet("tvp-var.jl")

In [242]:
coef = generate_coeffs([0.2 0.4], [1. 1.], [0.001 0.001], [0.01 0.02], 80)

round.(coef, digits=4)

80×1×2 Array{Float64, 3}:
[:, :, 1] =
 0.2025
 0.1926
 0.1979
 0.1966
 0.2143
 0.22
 0.2145
 0.2379
 0.2386
 0.2487
 ⋮
 0.3051
 0.3047
 0.2835
 0.289
 0.2771
 0.2713
 0.2918
 0.303
 0.3005

[:, :, 2] =
 0.4042
 0.4004
 0.4081
 0.426
 0.4019
 0.3977
 0.408
 0.4097
 0.4327
 0.4214
 ⋮
 0.2854
 0.288
 0.3025
 0.326
 0.3517
 0.3523
 0.3459
 0.3171
 0.2924

In [243]:
round.(rand(Uniform(-1,1),2), digits=2)

2-element Vector{Float64}:
  0.8
 -0.91

In [244]:
#init params
B0 = [0.6, 0.4]
P0 = diagm([.05, .03])
ν = diagm([.015, .01])
Y = .035
X = [0.2, 0.1]'
σ = 0.04


S = measurement_cov(X, P0, σ)
display(S)
K = k_gain(X,ν, S)
display(K)
res = residuals(Y, X, B0)
display(res)
states = update_state(B0, K, res)
display(states)
filtered_cov = update_cov(P0, K, X) 
display(filtered_cov)
#[0. 0.] + (K * res)' 


0.042300000000000004

2-element Vector{Float64}:
 0.07092198581560283
 0.02364066193853428

-0.125

2-element Vector{Float64}:
 0.5911347517730496
 0.39704491725768326

2×2 Matrix{Float64}:
  0.0492908    -0.000212766
 -0.000236407   0.0299291

In [245]:
B0 = [0.6, 0.4]
P0 = diagm([.05, .03])
ν = diagm([.015, .01])
Y = .035
X = [0.2, 0.1]
σ = 0.04

res = kalman_step2(B0,P0, Y, X, σ, ν)

display(round.(res[1], digits=4))
display(round.(res[2], digits=4))

2-element Vector{Float64}:
 0.5622
 0.3884

2×2 Matrix{Float64}:
  0.0611  -0.0012
 -0.0012   0.0396

In [260]:
X1 = [.54 .92]


X_test = round.(sample(X1, coef, .09, 80)', digits=2)[2:end,:]

80×2 Matrix{Float64}:
  0.2    0.13
  0.09   0.08
  0.07  -0.19
  0.1   -0.03
  0.0   -0.02
  0.15   0.05
  0.08  -0.02
 -0.14   0.15
  0.08   0.09
 -0.04  -0.04
  ⋮     
 -0.01  -0.12
 -0.16  -0.2
 -0.15  -0.08
  0.03   0.08
 -0.0    0.23
  0.13   0.18
  0.07   0.19
 -0.04   0.06
 -0.08  -0.02

In [261]:
Y = X_test[2:end,1]

79-element Vector{Float64}:
  0.09
  0.07
  0.1
  0.0
  0.15
  0.08
 -0.14
  0.08
 -0.04
 -0.06
  ⋮
 -0.01
 -0.16
 -0.15
  0.03
 -0.0
  0.13
  0.07
 -0.04
 -0.08

In [272]:
display(X_test[1, :])
display(Y[1])


2-element Vector{Float64}:
 0.2
 0.13

0.09

In [273]:
B0 = [0.0, 0.0]
P0 = diagm([.00, .00])
ν = diagm([.01, .02])
σ = 0.09
res = kalmanFilter2(X_test[1:end-1,:], Y, B0, P0, σ, ν)

display(round.(res[1], digits=4))
display("cov")
display(round.(res[2], digits=4))

80×2 Matrix{Float64}:
  0.0      0.0
  0.002    0.0026
  0.0034   0.005
  0.0057  -0.0074
  0.0056  -0.0074
  0.0056  -0.0106
  0.0134  -0.0055
  0.0048  -0.0011
 -0.0045   0.019
 -0.0078   0.012
  ⋮       
  0.1441  -0.1859
  0.1682  -0.0856
  0.2471   0.0282
  0.2144   0.0057
  0.2137   0.0034
  0.2137   0.1082
  0.2224   0.1212
  0.2093   0.0755
  0.2214   0.059

"cov"

80×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.0      0.0
 0.01    -0.0001
 0.0199  -0.0001
 0.0299   0.0001
 0.0397   0.0002
 0.0497   0.0002
 0.0588  -0.0003
 0.0685  -0.0002
 0.0772   0.0026
 0.0866   0.0013
 ⋮       
 0.3138   0.0803
 0.3221   0.0731
 0.294    0.0181
 0.2818   0.0029
 0.2909  -0.0001
 0.3009  -0.0001
 0.2955  -0.0232
 0.3027  -0.0331
 0.3104  -0.0299

[:, :, 2] =
  0.0     0.0
 -0.0001  0.0199
 -0.0001  0.0398
  0.0001  0.0584
  0.0002  0.0784
  0.0002  0.0983
 -0.0003  0.1179
 -0.0002  0.1378
  0.0026  0.1519
  0.0013  0.1693
  ⋮       
  0.0803  0.4197
  0.0731  0.41
  0.0181  0.3507
  0.0029  0.3602
 -0.0001  0.3702
 -0.0001  0.3174
 -0.0232  0.3027
 -0.0331  0.2879
 -0.0299  0.3036

In [274]:
s = smooth_state([0.28, 0.09], [0.2, .05], diagm([0.04, 0.08]), diagm([0.01, 0.02]))

round.(s, digits=4)

2-element Vector{Float64}:
 0.264
 0.082

In [275]:
c = smooth_cov(diagm([0.04, 0.08]), diagm([0.01, 0.02]))

2×2 Matrix{Float64}:
 0.008  0.0
 0.0    0.016

In [276]:
S_filtered = res[1]
P_filtered = res[2]

sampled = simulation_smoother(S_filtered, P_filtered, ν)

round.(sampled, digits=4)

80×2 Matrix{Float64}:
  0.0      0.0
 -0.0601   0.0323
  0.0252  -0.0679
  0.0658  -0.2741
 -0.0409  -0.1932
  0.0731  -0.1208
  0.1292   0.0857
  0.021    0.1077
  0.0171   0.2395
  0.1294   0.2381
  ⋮       
  0.6143   0.2032
  0.3777   0.1563
  0.4156   0.148
  0.4285   0.3407
  0.3727   0.1104
  0.2734   0.2851
  0.2376   0.5121
  0.1624   0.6622
  0.1701   0.4014

In [277]:
n_draws = 10_000

coeff_post = zeros(n_draws,80, 2)

for d in 1:n_draws
    coeff_post[d, :, :] = carter_kohn(X_test[1:end-1,:], Y, B0, P0, σ, ν)
end

In [282]:
mean(coeff_post[:,3,:], dims=1)

1×2 Matrix{Float64}:
 0.0161428  0.00270553